# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

pd.options.mode.chained_assignment = None

import seaborn as sns
import os


In [2]:
# загрузка данных
if os.path.exists('/datasets/insurance.csv'):
    df = pd.read_csv('/datasets/insurance.csv')

else:
    df = pd.read_csv('/Users/olesya/datasets/insurance.csv')


In [3]:
df.head()

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Проверим данные на дубликаты.

In [5]:
df.duplicated().sum()

153

Итак: данные состоят из 5 колонок и 5000 строк, пропусков нет, но имеется 153 дубликата.
Избавимся от дубликатов, так как совпадение по всем признакам крайне маловероятно.

In [6]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Постараемся ответить на вопрос:**

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

Чтобы доказать изменится ли качество линейной регрессии, нужно сравнить равенства 
$
a = Xw
$
и 
$
a_z = Zw_z
$. Если равенства равны, то качество не изменится.

Представим нашу новую матрицу признаков (Z) как результат умножения исходной матрицы признаков (Х) на некую обратимую матрицу (Р):

$$
Z = XP
$$

Тогда формула обучения примет вид:

$$
w_z = (Z^T Z)^{-1} Z^T y
$$

Теперь подставим в выражение значение Z:

$$
w_z = ((XP)^T (XP))^{-1} (XP)^T y
$$

Раскроем скобки:

$$
w_z = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Воспользуемся следующими свойствами матриц:

$$
A A^{-1} = A^{-1} A = E =>   (P^T)^{-1} P^T = E
$$

$$
AE = EA = A => E X^T = X^T
$$

Получим:

$$
w_z = P^{-1} w
$$

Подставим получившиеся значения в формулу предсказания: $𝑎_z = 𝑍 𝑤_𝑧$, где $Z = XP$,  $w_z = P^{-1} w$, получим:

$$
𝑎_z = XP P^{-1} w = Xw
$$

Тем самым доказали, что $a_z = a$

**Ответ:** Следовательно качество линенйной регрессии не изменится.

## Алгоритм преобразования

**Алгоритм**

- Создадим случайную обратимую матрицу P, размерностью соответствующую количеству столбцов признаков (4х4). 
- Затем умножим матрицу с признаками X на матрицу P. 
- Посчитаем качество модели на полученной матрице и сравним с качеством модели на матрице исходных данных.


**Обоснование**

Как было доказано в пункте выше, качество линейной регрессии не изменится при умножении матрицы признаков на любую обратимую матрицу P. Поэтому можно следовать описанному выше алгоритму без опасения получить другой результат линейной регрессии.

## Проверка алгоритма

Разделим данные на набор признаков и на целевой признакё

In [7]:
# создание переменных для признаков и целевого признака
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

Сгенерируем случайную матрицу размером 4х4 функцией numpy.random.normal(), в этом случае вероятность получить необратимую матрицу близка к нулю.

In [8]:
np.random.seed(42)
random_matrix = np.random.normal(size=(4, 4))

print(random_matrix)

[[ 0.49671415 -0.1382643   0.64768854  1.52302986]
 [-0.23415337 -0.23413696  1.57921282  0.76743473]
 [-0.46947439  0.54256004 -0.46341769 -0.46572975]
 [ 0.24196227 -1.91328024 -1.72491783 -0.56228753]]


Проверим матрицу на обратимость методом linalg.inv(). При использовании необратимой матрицы дальнейшая работа по описанному алгоритму невозможна. Если появится ошибка, то наша случайная матрица необратима.

In [9]:
np.linalg.inv(random_matrix)

array([[-0.33124857, -1.0333483 , -2.21070923, -0.47651024],
       [ 0.12864778, -0.47781825,  0.15271687, -0.43017975],
       [-0.51338817,  0.33588835, -0.8347692 , -0.24072429],
       [ 0.99462192,  0.15079316,  1.08985145,  0.21872535]])

Отлично, созданная случайная матрица - обратима, значит идем дальше.

Создадим матрицу из исходных признаков

In [10]:
matrix = features.values
print(matrix)

[[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00]]


Перемножим исходную матрицу на сгенерированную matrix * random_matrix

In [11]:
# перемножение исходной матриы на новую сгенерированную матрицу
matrix_new = matrix @ random_matrix
# перевод матрицы в датафрейм и вывод на экран
features_new = pd.DataFrame(matrix_new, columns=features.columns)
display(features_new)

Пол       Возраст      Зарплата   Члены семьи
0    -23294.791154  26899.327002 -22921.847067 -23067.770211
1    -17850.555758  20604.598076 -17538.953455 -17662.990926
2     -9865.752553  11386.970944  -9685.974377  -9758.069218
3    -19581.515190  22616.010381 -19294.804157 -19405.939170
4    -12259.341053  14154.123038 -12050.336135 -12132.535366
...            ...           ...           ...           ...
4842 -16766.307948  19359.011161 -16503.243510 -16606.188605
4843 -24608.177075  28420.272347 -24231.118785 -24378.708594
4844 -15919.380826  18384.276178 -15681.725366 -15774.014527
4845 -15355.741193  17730.684307 -15123.542938 -15212.643210
4846 -19066.477687  22019.330390 -18771.617599 -18886.179080

[4847 rows x 4 columns]

Создадим класс LinearRegression для последующего обучения наших моделей

In [12]:
# создание класса LinearRegression для обучения модели
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@ X)@ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0  

Обучим модель и проверим качество модели на исходных данных

In [13]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)

print(r2_score(target, predictions))

0.4302010044852067


Обучим модель и проверим качество модели на закодированных данных

In [14]:
model = LinearRegression()
model.fit(features_new, target)
predictions = model.predict(features_new)

print(r2_score(target, predictions))

0.43020100448520104


<b>Вывод</b>

Проверили качество модели на исходной матрице и закодированной. Таким образом доказали, что качество линейной регрессии не меняется при умножении матрицы признаков на любую обратимую матрицу P.